In [50]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [51]:
import PrjtCF_module as cf

# Outline
* 호텔, 골프장 등 개발 후 운영 사업(분양을 하지 않는 사업)의 개발단계 재무모델
* PF대출을 통해 토지비 및 공사비 등 개발사업 자금을 조달
* 금융비용은 PF대출금으로 충당
* 준공 후 담보대출금으로 상환 가정
* PF대출금은 필요에 따라 한도대로 인출
* PF대출금의 조달을 Tranche A, Tranche B 두개의 Tranche로 구분하여 조달하며, Tranche A는 Tranche B에 대하여 선순위 상환 받음.
* 기존 작성한 Cash Flow 모델에서 추가적으로 index 설정을 loan에 대해서도 동일한 기준을 적용.
  - loan index는 별도 금융조건 계산에만 적용

# Input Index Data

In [52]:
# 기간 index 설정
# prjt index : 사업기간 전 기간에 대한 index로 모델의 base가 되는 index
# cstrn index : 공사기간에 대한 index로 착공일로부터 준공일 까지의 기간에 대한 index
# loan index : 대출기간에 대한 index로 대출 실행일로부터 대출 상환일 까지의 기간에 대한 index
idx = cf.PrjtIndex(idxname=['prjt', 'cstrn', 'loan'],
                   start=['2021-08', '2021-10', '2021-10'],
                   periods=[24+1, 18+1, 20+1],
                   freq='M')

# 공정률 index 설정 : cstrn index 기간 중 적용되는 공정률을 설정
idx.prcs = Series(np.ones(len(idx.cstrn)) / len(idx.cstrn),
                  index=idx.cstrn.index)

# Input Financing Data

In [53]:
equity = cf.Loan(idx, idx, amt_ntnl=10_000)

tra = cf.Loan(idx, idx.loan, amt_ntnl=40_000,
              rate_fee=0.01, rate_IR=0.05)
trb = cf.Loan(idx, idx.loan, amt_ntnl=20_000,
              rate_fee=0.03, rate_IR=0.07)

# Input Cost Data

In [54]:
dct_cost = {}

# 토지비: 최초 1회 지급
lnd = cf.Account(idx)
lnd.addscdd(idx.cstrn[0], 10_000)
dct_cost['lnd'] = lnd

# 공사비: 공정률에 따라 지급
cstrn = cf.Account(idx)
cstrn.addscdd(idx.cstrn.index, 50_000 * idx.prcs)
dct_cost['cstrn'] = cstrn

cost = cf.Merge(dct_cost)

# Execution Cash Flow

### 1) 사전 설정

In [55]:
# Make accounts
oprtg = cf.Account(idx) # 운영현금흐름의 입출을 위한 운영계좌
rpyacc = cf.Account(idx) # 상환자금 관리를 위한 대출금 상환계좌

In [56]:
# Calculate expected financial cost
class exptdcst:
    def __init__(self, idxno):
        self.idxno = idxno
        
        # 해당 인덱스 기간 중 cost계좌 상 지출 필요한 금액
        self.cstamt = cost.add_scdd[idxno]
        
        # 해당 인덱스 기간 중 loan계좌 상 지출 필요한 수수료 금액
        self.trafee = tra.fee.add_scdd[idxno]
        self.trbfee = trb.fee.add_scdd[idxno]
        
        # 해당 인덱스 기간 중 loan계좌 상 지출 필요한 이자 금액
        self.traIR = -tra.ntnl.bal_strt[idxno] * tra.IR.rate
        self.trbIR = -trb.ntnl.bal_strt[idxno] * trb.IR.rate
        
    @property
    def ttlcst(self):
        self.ttlsum = self.cstamt
        self.ttlsum = (self.ttlsum +
                       self.trafee + self.trbfee +
                       self.traIR + self.trbIR)
        return self.ttlsum

In [57]:
# Calculate expected repayment of loan
class exptdrpy:
    def __init__(self, idxno):
        self.idxno = idxno
        
        # 해당 인덱스 기간 중 상환 요구 금액
        self.trarpy = min(tra.ntnl.add_rsdl_cum[idxno], -tra.ntnl.bal_end[idxno])
        self.trbrpy = min(trb.ntnl.add_rsdl_cum[idxno], -trb.ntnl.bal_end[idxno])
    
    @property
    def ttlrpy(self):
        self.ttlsum = self.trarpy + self.trbrpy
        return self.ttlsum

### 2) 실행함수 설정

In [66]:
# Inflow of cash
"""
def inflow_cash(idxno, exptdcst):
    #### Inflow of equity amount ####
    equity.ntnl.send(idxno, equity.ntnl.sub_scdd[idxno], oprtg)
    
    #### Inflow of loan notional amount ####
    if idxno in idx.loan:
        # Expected inflow amount: 예상 지출금액을 인자로 받아 운영계좌 잔액과 비교하여 필요한 조달금액을 추정
        exptdinflw = max(exptdcst.ttlcst - oprtg.bal_end[idxno], 0)
    
        # Tr.B: 필요 조달금액을 잔여 대출금 한도와 비교하여 실제 조달되는 금액 계산
        tmpinflw = min(trb.ntnl.sub_rsdl_cum[idxno], exptdinflw)
        trb.ntnl.send(idxno, tmpinflw, oprtg) # 운영계좌 입금 처리
        amtinflw = tmpinflw
        exptdinflw -= tmpinflw
        
        # Tr.A: 필요 조달금액을 잔여 대출금 한도와 비교하여 실제 조달되는 금액 계산
        tmpinflw = min(tra.ntnl.sub_rsdl_cum[idxno], exptdinflw)
        tra.ntnl.send(idxno, tmpinflw, oprtg) # 운영계좌 입금 처리
        amtinflw += tmpinflw
        exptdinflw -= tmpinflw
"""
        
class Inflw_cash:
    def __init__(self, idxno, exptdcst):
        self.idxno = idxno
        self.exptdcst = exptdcst
        
        self.amt_inflw = 0
        self.rsdl_inflw = self.inflw_exptd
        
    def inflw_equity(self, eqty):
        amt_inflw = eqty.ntnl.sub_scdd[idxno]
        eqty.ntnl.send(idxno, amt_inflw, oprtg)
    
    def inflw_loan(self, loan):
        if idxno in idx.loan:
            tmp_inflw = min(loan.ntnl.sub_rsdl_cum[idxno], self.rsdl_inflw)
            loan.ntnl.send(idxno, tmp_inflw, oprtg) # 운영계좌에 입금 처리
            self.amt_inflw += tmp_inflw
            self.rsdl_inflw -= tmp_inflw
    
    @property
    def inflw_exptd(self):
        amtneed = max(self.exptdcst.ttlcst - oprtg.bal_end[idxno], 0)
        return amtneed

In [67]:
# Outflow of cash
def outflow_cash(idxno):
    # 운영계좌에서 토지비 지출
    oprtg.send(idxno, cost['lnd'].add_scdd[idxno], cost['lnd'])
    
    # 운영계좌에서 공사비 지출
    oprtg.send(idxno, cost['cstrn'].add_scdd[idxno], cost['cstrn'])

In [68]:
# Pay financial cost
def pay_fnclcst(idxno, exptdcst):
    # 운영계좌에서 대출금 수수료 지출
    oprtg.send(idxno, exptdcst.trafee, tra.fee)
    oprtg.send(idxno, exptdcst.trbfee, trb.fee)
        
    # 운영계좌에서 이자금액 지출
    oprtg.send(idxno, exptdcst.traIR, tra.IR)
    oprtg.send(idxno, exptdcst.trbIR, trb.IR)

In [69]:
# Transfer amount to repayment account
def trsf_rpy(idxno, exptdrpy):
    # 상환계좌로 이체할 금액 계산
    rpy_amt = exptdrpy.ttlrpy
    
    # 상환계좌로 이체
    oprtg.send(idxno, rpy_amt, rpyacc)

In [70]:
# Repay notional amount
def repay_ntnl(idxno, exptdrpy):
    # 상환계좌에서 순서에 따라 상환금 인출
    rpyacc.send(idxno, exptdrpy.trarpy, tra.ntnl)
    rpyacc.send(idxno, exptdrpy.trbrpy, trb.ntnl)

### 3) Cash Flow 실행

In [71]:
# Execute cash flow
for idxno in idx.index:
    # 해당 인덱스 기간 중 지출 필요한 cost 및 금융비용 계산(instance class 생성)
    cst_exptd = exptdcst(idxno)
    
    # 필요지출금액을 인자로 받아, 조달금액 계산 후 운영계좌 입금
    #inflow_cash(idxno, cst_exptd)
    inflw = Inflw_cash(idxno, cst_exptd)
    inflw.inflw_equity(equity)
    inflw.inflw_loan(tra)
    inflw.inflw_loan(trb)
    
    # 운영계좌에서 토지비, 공사비 등 각종 비용 지출
    outflow_cash(idxno)
    
    # 운영계좌에서 각종 금융비용 지출
    pay_fnclcst(idxno, cst_exptd)
    
    # 해당 인덱스 기간 중 상환 필요한 자금 규모 계산(instance class 생성)
    rpy_exptd = exptdrpy(idxno)
    
    # 상환 예정 금액을 상환계좌로 이체
    trsf_rpy(idxno, rpy_exptd)
    
    # 상환계좌에서 순서에 따라 상환금 인출
    repay_ntnl(idxno, rpy_exptd)

In [72]:
# Merge Accounts
dct_acc = {'equity':equity,
           'tra_ntnl':tra.ntnl,
           'tra_fee':tra.fee,
           'tra_IR':tra.IR,
           'trb_ntnl':trb.ntnl,
           'trb_fee':trb.fee,
           'trb_IR':trb.IR,
           'cost_lnd':cost['lnd'],
           'cost_cstrn':cost['cstrn'],
           'oprtg':oprtg}
acc_merge = cf.Merge(dct_acc)

In [73]:
# 현금흐름 종합
rslt_df = DataFrame({('equity.ntnl', 'amt_sub'):equity.ntnl.amt_sub[:],
                     ('tra.ntnl', 'amt_sub'):tra.ntnl.amt_sub[:],
                     ('tra.ntnl', 'amt_add'):tra.ntnl.amt_add[:],
                     ('tra.ntnl', 'bal_end'):tra.ntnl.bal_end[:],
                     ('tra.fee', 'amt_add'):tra.fee.amt_add[:],
                     ('tra.IR', 'amt_add'):tra.IR.amt_add[:],
                     ('trb.ntnl', 'amt_sub'):trb.ntnl.amt_sub[:],
                     ('trb.ntnl', 'amt_add'):trb.ntnl.amt_add[:],
                     ('trb.ntnl', 'bal_end'):trb.ntnl.bal_end[:],
                     ('trb.fee', 'amt_add'):trb.fee.amt_add[:],
                     ('trb.IR', 'amt_add'):trb.IR.amt_add[:],
                     ('cost_lnd', 'amt_add'):cost['lnd'].amt_add[:],
                     ('cost_cstrn', 'amt_add'):cost['cstrn'].amt_add[:],
                     ('oprtg', 'amt_add'):oprtg.amt_add[:],
                     ('oprtg', 'amt_sub'):oprtg.amt_sub[:],
                     ('oprtg', 'bal_end'):oprtg.bal_end[:]})
rslt_df.fillna(0).applymap(lambda x: f"{x:,.0f}")

equity.ntnl tra.ntnl                  tra.fee  tra.IR trb.ntnl  \
               amt_sub  amt_sub amt_add  bal_end amt_add amt_add  amt_sub   
2021-08-31      10,000        0       0        0       0       0        0   
2021-09-30           0        0       0        0       0       0        0   
2021-10-31           0    3,632       0   -3,632     400       0        0   
2021-11-30           0    2,647       0   -6,278       0      15        0   
2021-12-31           0    2,658       0   -8,936       0      26        0   
2022-01-31           0    2,669       0  -11,605       0      37        0   
2022-02-28           0    2,680       0  -14,285       0      48        0   
2022-03-31           0    2,691       0  -16,976       0      60        0   
2022-04-30           0    2,702       0  -19,678       0      71        0   
2022-05-31           0    2,714       0  -22,392       0      82        0   
2022-06-30           0    2,725       0  -25,117       0      93        0   
2022-07-31           0    2,736       0  -27,853       0     105        0   
2022-08-31           0    2,748       0  -30,600       0     116        0   
2022-09-30           0    2,759       0  -33,360       0     128        0   
2022-10-31           0    2,771       0  -36,130       0     139        0   
2022-11-30           0    2,782       0  -38,912       0     151        0   
2022-12-31           0    1,088       0  -40,000       0     162    1,706   
2023-01-31           0        0       0  -40,000       0     167    2,808   
2023-02-28           0        0       0  -40,000       0     167    2,825   
2023-03-31           0        0       0  -40,000       0     167    2,841   
2023-04-30           0        0       0  -40,000       0     167    2,858   
2023-05-31           0        0       0  -40,000       0     167      243   
2023-06-30           0        0  40,000        0       0     167      244   
2023-07-31           0        0       0        0       0       0        0   
2023-08-31           0        0       0        0       0       0        0   

                            trb.fee  trb.IR cost_lnd cost_cstrn   oprtg  \
           amt_add  bal_end amt_add amt_add  amt_add    amt_add amt_add   
2021-08-31       0        0       0       0        0          0  10,000   
2021-09-30       0        0       0       0        0          0       0   
2021-10-31       0        0     600       0   10,000      2,632   3,632   
2021-11-30       0        0       0       0        0      2,632   2,647   
2021-12-31       0        0       0       0        0      2,632   2,658   
2022-01-31       0        0       0       0        0      2,632   2,669   
2022-02-28       0        0       0       0        0      2,632   2,680   
2022-03-31       0        0       0       0        0      2,632   2,691   
2022-04-30       0        0       0       0        0      2,632   2,702   
2022-05-31       0        0       0       0        0      2,632   2,714   
2022-06-30       0        0       0       0        0      2,632   2,725   
2022-07-31       0        0       0       0        0      2,632   2,736   
2022-08-31       0        0       0       0        0      2,632   2,748   
2022-09-30       0        0       0       0        0      2,632   2,759   
2022-10-31       0        0       0       0        0      2,632   2,771   
2022-11-30       0        0       0       0        0      2,632   2,782   
2022-12-31       0   -1,706       0       0        0      2,632   2,794   
2023-01-31       0   -4,514       0      10        0      2,632   2,808   
2023-02-28       0   -7,339       0      26        0      2,632   2,825   
2023-03-31       0  -10,180       0      43        0      2,632   2,841   
2023-04-30       0  -13,037       0      59        0      2,632   2,858   
2023-05-31       0  -13,280       0      76        0          0     243   
2023-06-30  13,524       -0       0      77        0          0     244   
2023-07-31       0        0       0       0        0    

In [74]:
# 현금흐름 요약
rslt_df = DataFrame({('equity.ntnl', 'amt_sub'):equity.ntnl.amt_sub[:],
                     ('tra.ntnl', 'amt_sub'):tra.ntnl.amt_sub[:],
                     ('tra.ntnl', 'amt_add'):tra.ntnl.amt_add[:],
                     ('tra.fee', 'amt_add'):tra.fee.amt_add[:],
                     ('tra.IR', 'amt_add'):tra.IR.amt_add[:],
                     ('trb.ntnl', 'amt_sub'):trb.ntnl.amt_sub[:],
                     ('trb.ntnl', 'amt_add'):trb.ntnl.amt_add[:],
                     ('trb.fee', 'amt_add'):trb.fee.amt_add[:],
                     ('trb.IR', 'amt_add'):trb.IR.amt_add[:],
                     ('cost_lnd', 'amt_add'):cost['lnd'].amt_add[:],
                     ('cost_cstrn', 'amt_add'):cost['cstrn'].amt_add[:],
                     ('oprtg', 'amt_add'):oprtg.amt_add[:],
                     ('oprtg', 'amt_sub'):oprtg.amt_sub[:],
                     ('oprtg', 'bal_end'):oprtg.bal_end[:]})
rslt_df.fillna(0).applymap(lambda x: f"{x:,.0f}")

equity.ntnl tra.ntnl         tra.fee  tra.IR trb.ntnl          \
               amt_sub  amt_sub amt_add amt_add amt_add  amt_sub amt_add   
2021-08-31      10,000        0       0       0       0        0       0   
2021-09-30           0        0       0       0       0        0       0   
2021-10-31           0    3,632       0     400       0        0       0   
2021-11-30           0    2,647       0       0      15        0       0   
2021-12-31           0    2,658       0       0      26        0       0   
2022-01-31           0    2,669       0       0      37        0       0   
2022-02-28           0    2,680       0       0      48        0       0   
2022-03-31           0    2,691       0       0      60        0       0   
2022-04-30           0    2,702       0       0      71        0       0   
2022-05-31           0    2,714       0       0      82        0       0   
2022-06-30           0    2,725       0       0      93        0       0   
2022-07-31           0    2,736       0       0     105        0       0   
2022-08-31           0    2,748       0       0     116        0       0   
2022-09-30           0    2,759       0       0     128        0       0   
2022-10-31           0    2,771       0       0     139        0       0   
2022-11-30           0    2,782       0       0     151        0       0   
2022-12-31           0    1,088       0       0     162    1,706       0   
2023-01-31           0        0       0       0     167    2,808       0   
2023-02-28           0        0       0       0     167    2,825       0   
2023-03-31           0        0       0       0     167    2,841       0   
2023-04-30           0        0       0       0     167    2,858       0   
2023-05-31           0        0       0       0     167      243       0   
2023-06-30           0        0  40,000       0     167      244  13,524   
2023-07-31           0        0       0       0       0        0       0   
2023-08-31           0        0       0       0       0        0       0   

           trb.fee  trb.IR cost_lnd cost_cstrn   oprtg                   
           amt_add amt_add  amt_add    amt_add amt_add amt_sub  bal_end  
2021-08-31       0       0        0          0  10,000       0   10,000  
2021-09-30       0       0        0          0       0       0   10,000  
2021-10-31     600       0   10,000      2,632   3,632  13,632        0  
2021-11-30       0       0        0      2,632   2,647   2,647        0  
2021-12-31       0       0        0      2,632   2,658   2,658        0  
2022-01-31       0       0        0      2,632   2,669   2,669        0  
2022-02-28       0       0        0      2,632   2,680   2,680        0  
2022-03-31       0       0        0      2,632   2,691   2,691        0  
2022-04-30       0       0        0      2,632   2,702   2,702        0  
2022-05-31       0       0        0      2,632   2,714   2,714        0  
2022-06-30       0       0        0      2,632   2,725   2,725        0  
2022-07-31       0       0        0      2,632   2,736   2,736        0  
2022-08-31       0       0        0      2,632   2,748   2,748        0  
2022-09-30       0       0        0      2,632   2,759   2,759        0  
2022-10-31       0       0        0      2,632   2,771   2,771        0  
2022-11-30       0       0        0      2,632   2,782   2,782        0  
2022-12-31       0       0        0      2,632   2,794   2,794        0  
2023-01-31       0      10        0      2,632   2,808   2,808        0  
2023-02-28       0      26        0      2,632   2,825   2,825        0  
2023-03-31       0      43        0      2,632   2,841   2,841        0  
2023-04-30       0      59        0      2,632   2,858   2,858        0  
2023-05-31       0      76        0          0     243     243        0  
2023-06-30       0      77        0          0     244  53,768  -53,524  
2023-07-31       0       0        0          0       0       0  -53,524  
2023-08-31       0       0        